In [ ]:
from typing import Dict
import uvicorn
from fastapi import Depends, FastAPI
from pydantic import BaseModel
import os

#app = FastAPI()
app = FastAPI(openapi_prefix='/model-serving/'+os.getenv("HOSTNAME").split('-')[0])


class SentimentRequest(BaseModel):
    text: str


class SentimentResponse(BaseModel):
    probabilities: Dict[str, float]
    sentiment: str
    confidence: float

class HeartbeatResult(BaseModel):
    is_alive: bool

@app.post("/predict", response_model=SentimentResponse)
def predict(request: SentimentRequest, model: Model = Depends(get_model)):
    sentiment, confidence, probabilities = model.predict(request.text)
    return SentimentResponse(
        sentiment=sentiment, confidence=confidence, probabilities=probabilities
    )

@app.get("/", response_model=HeartbeatResult)
def get_heartbeat()-> HeartbeatResult:
    heartbeat = HeartbeatResult(is_alive=True)
    return heartbeat

if __name__ == "__main__":
    uvicorn.run(app, log_level="info")

In [2]:
!pip install fastapi uvicorn pydantic

You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from typing import Dict, List
import uvicorn
from fastapi import Depends, FastAPI
from pydantic import BaseModel
import os
from aitextgen import aitextgen
import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )
#app = FastAPI()
app = FastAPI(openapi_prefix='/model-serving/'+os.getenv("HOSTNAME").split('-')[0])
ai = aitextgen(model="trained_model/pytorch_model.bin", config="trained_model/config.json", to_gpu=True)

def get_model():
    return ai

class GPT2Request(BaseModel):
    promt: str
    samples: int = 5
    batch_size: int = 5
    max_length: int = 256
    temperature: int = 1.0


class GPT2Response(BaseModel):
    generated: List[str]

class HeartbeatResult(BaseModel):
    is_alive: bool


@app.post("/predict", response_model=GPT2Response)
def predict(request: GPT2Request, model = Depends(get_model)):
    response = ai.generate(n=request.samples,
            batch_size=request.samples,
            prompt=request.promt,
            max_length=256,
            temperature=1.0,
            top_p=0.9,
            return_as_list=True)
    
    # Bold the prompt if printing to console
    gen_texts = [
            text.replace(prompt_text, f"\033[1m{prompt_text}\033[0m", 1)
            for text in gen_texts
    ]
    print(*gen_texts, sep="\n" + "=" * 10 + "\n")
    return GPT2Response(
        generated=response
    )

@app.get("/", response_model=HeartbeatResult)
def get_heartbeat()-> HeartbeatResult:
    heartbeat = HeartbeatResult(is_alive=True)
    return heartbeat

if __name__ == "__main__":
    uvicorn.run(app, log_level="debug")

INFO:aitextgen:Loading GPT-2 model from provided trained_model/pytorch_model.bin.
INFO:aitextgen:Using the default GPT-2 Tokenizer.


RuntimeError: This event loop is already running

INFO:     Started server process [17117]
INFO:uvicorn.error:Started server process [17117]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


In [ ]:
uvicorn.run(app, log_level="debug")

In [ ]:
!kill -9 16935